In [0]:
from pyspark.sql import functions as F

In [0]:
schema_path = "dbfs:/FileStore/inventory_analytics"

In [0]:
purchase_df = spark.read.format("delta").load(f"{schema_path}/silver/product_purchase")

In [0]:
daily_hourly_purchase = purchase_df\
    .withColumn("days_ago", F.date_diff(F.current_timestamp(), F.col("timestamp")))\
    .filter("days_ago < 14")\
    .withColumn("hourly_timestamp", F.date_trunc("hour", F.col("timestamp")))\
    .groupBy("pin_code", "product_id", "hourly_timestamp")\
    .agg(F.sum("item_count").alias("purchase_count"))

In [0]:
weighted_hourly_avg_purchase_next_24_hour = daily_hourly_purchase\
    .withColumn("days_ago", F.date_diff(F.current_timestamp(), F.col("hourly_timestamp")))\
    .withColumn("weight", 1-(F.col("days_ago")/14))\
    .withColumn("weighted_purchase_count", F.col("weight")*F.col("purchase_count"))\
    .withColumn("hour_of_day", F.date_part(F.lit("hour"), F.col("hourly_timestamp"), ))\
    .groupBy("pin_code", "product_id", "hour_of_day")\
    .agg(
        F.sum("weighted_purchase_count").alias("total_weighted_purchase_count"),
        F.sum("weight").alias("total_weight")
    )\
    .withColumn("weighted_avg_purchase", F.col("total_weighted_purchase_count") / F.col("total_weight"))\
    

In [0]:
weighted_hourly_avg_purchase_next_24_hour.write.format("delta").option("path", f"{schema_path}/gold/purchase_count").save()

### click ratio

In [0]:
click_df = spark.read.format("delta").load(f"{schema_path}/silver/product_click")

In [0]:
daily_hourly_click_last_14_days = click_df\
    .withColumn("days_ago", F.date_diff(F.current_date(), F.col("timestamp")))\
    .filter("days_ago < 14")\
    .withColumn("hourly_timestamp", F.date_trunc("hour", F.col("timestamp")))\
    .groupBy("pin_code", "product_id", "hourly_timestamp")\
    .agg(F.count("*").alias("click_count"))

In [0]:
weighted_hourly_avg_click_14_days = daily_hourly_click_last_14_days\
    .withColumn("days_ago", F.date_diff(F.current_timestamp(), "hourly_timestamp"))\
    .withCoulmn("weight", 1-(F.col("days_ago")/14))\
    .withColumn("avg_weighted_click", F.col("click_count")*F.col("weight"))\
    .withColumn("hour_of_day", F.date_part(F.lit("hour"), F.col("hourly_timestamp")))\
    .groupBy("pin_code", "product_id", "hour_of_day")\
    .agg(
        F.sum("weighted_click_count").alias("total_weighted_click_count"),
        F.sum("weight").alias("total_weight")
    )\
    .withColumn("weighted_avg_click", F.col("total_weighted_click_count") / F.col("total_weight"))

In [0]:
click_count_last_1_day = click_df\
    .filter("hourly_timestamp" <= F.current_timestamp() - F.expr("INTERVAL 24 HOUR"))\
    .withColumn("hour_of_day", F.date_part(F.lit("hour"), F.col("hourly_timestamp")))\
    .groupBy("pin_code", "product_id", "hour_of_day")\
    .agg(F.count("*").alias("click_count"))

In [0]:
click_ratio = click_count_last_1_day.join(
    weighted_hourly_avg_click_14_days.filter("weighted_avg_click > 0"),
    ["pin_code", "product_id", "hour_of_day"],
    "inner"
    ).withColumn("click_ratio",
                  click_count_last_1_day.click_count / weighted_hourly_avg_click_14_days.weighted_avg_click
                 )

In [0]:
click_ratio.write.format("delta").save(f"{schema_path}/gold/click_ratio")

### cart_ratio

In [0]:
cart_df = spark.read.format("delta").load(f"{schema_path}/silver/product_cart")

In [0]:
daily_hourly_cart_last_14_days = cart_df\
    .withColumn("days_ago", F.date_diff(F.current_date(), F.col("timestamp")))\
    .filter("days_ago < 14")\
    .withColumn("hourly_timestamp", F.date_trunc("hour", F.col("timestamp")))\
    .groupBy("pin_code", "product_id", "hourly_timestamp")\
    .agg(F.count("*").alias("cart_count"))

In [0]:
weighted_hourly_avg_cart_14_days = daily_hourly_cart_last_14_days\
    .withColumn("days_ago", F.date_diff(F.current_timestamp(), "hourly_timestamp"))\
    .withCoulmn("weight", 1-(F.col("days_ago")/14))\
    .withColumn("avg_weighted_click", F.col("cart_count")*F.col("weight"))\
    .withColumn("hour_of_day", F.date_part(F.lit("hour"), F.col("hourly_timestamp")))\
    .groupBy("pin_code", "product_id", "hour_of_day")\
    .agg(
        F.sum("weighted_cart_count").alias("total_weighted_cart_count"),
        F.sum("weight").alias("total_weight")
    )\
    .withColumn("weighted_avg_cart", F.col("total_weighted_cart_count") / F.col("total_weight"))

In [0]:
cart_count_last_1_day = cart_df\
    .filter("hourly_timestamp" <= F.current_timestamp() - F.expr("INTERVAL 24 HOUR"))\
    .withColumn("hour_of_day", F.date_part(F.lit("hour"), F.col("hourly_timestamp")))\
    .groupBy("pin_code", "product_id", "hour_of_day")\
    .agg(F.count("*").alias("cart_count"))

In [0]:
cart_ratio = cart_count_last_1_day.join(
    weighted_hourly_avg_click_14_days.filter("weighted_avg_cart > 0"),
    ["pin_code", "product_id", "hour_of_day"],
    "inner"
    ).withColumn("cart_ratio",
                cart_count_last_1_day.cart_count / weighted_hourly_avg_cart_14_days.weighted_avg_cart
                 )

In [0]:
cart_ratio.write.format("delta").save(f"{schema_path}/gold/cart_ratio")